In [ ]:
import setup  # notebooks/setup.py
setup.init()

In [ ]:
import uuid

In [ ]:
from mypermit import permit_client

permit = permit_client

In [ ]:
await permit.api.users.list()

In [ ]:
from django.contrib.auth import get_user_model

User = get_user_model()

In [ ]:
for u in User.objects.all():
    user_data = {
        "key": f"{u.id}",
        "email": f"{u.email}"
    }
    response = await permit.api.users.sync(user_data)
    print(response)

In [ ]:
await permit.check("user_id:1", "read", "document")

In [ ]:
resources = [
    {
        "name": "Document",
        "key": "document",
        "actions": {
            "read": {},
            "create": {},
            "update": {},
            "delete": {}
        }
    },
    {
        "name": "Movie Discovery Resource",
        "key": "movie_discovery",
        "actions": {
            "search": {},
            "detail": {}
        }
    }
]

In [ ]:
permissions = [
    "document:read",
    "movie_discovery:search",
    "movie_discovery:detail"
]

In [ ]:
roles = [
    {
        "name": "Manager",
        "key": "manager",
        "permissions": [
            "document:read",
            "document:create",
            "document:update",
            "document:delete",
            "movie_discovery:search",
            "movie_discovery:detail"
        ]
    },
    {
        "name": "Viewer",
        "key": "viewer",
        "permissions": [
            "document:read",
            "movie_discovery:search",
            "movie_discovery:detail"
        ]
    }
]

In [ ]:
from permit import PermitApiError

In [ ]:

async def get_or_create_resource(resource_data: dict):
    resource_key = resource_data.get("key")
    try:
        _resource = await permit.api.resources.get(resource_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.resources.create(resource_data)
        else:
            _resource = None
            raise
    return _resource

In [ ]:
# for resurce_data in resources:
#     response = await get_or_create_resource(resurce_data)
#     print(response)

In [ ]:


async def get_or_create_role(role_data: dict):
    _key = role_data.get("key")
    try:
        _resource = await permit.api.roles.get(_key)
    except PermitApiError as e:
        if e.status_code == 404:
            _resource = await permit.api.roles.create(role_data)
        else:
            _resource = None
            raise
    return _resource

In [ ]:
# for resurce_data in roles:
#     response = await get_or_create_role(resurce_data)
#     print(response)

In [28]:
user = User.objects.all().last()

In [32]:
response = await permit.api.users.assign_role({
    "user": f"{user.id}",
    "role": "viewer",
    "tenant": "default"
})
response

RoleAssignmentRead(id=UUID('3b4df538-7e5b-4332-9e81-aa70b666c906'), user='2', role='viewer', tenant='default', resource_instance=None, resource_instance_id=None, user_id=UUID('497632ef-ba9f-4004-bfae-7566a612b31e'), role_id=UUID('d5edc6a4-d720-4e82-b7ac-97d9097304b3'), tenant_id=UUID('4963f3ac-10e7-4e0c-a55b-71906765891e'), organization_id=UUID('370e006e-8ae9-4d9b-ac3e-6ff175a13598'), project_id=UUID('f8c773b5-c486-49a8-b77e-d2c880a31bcf'), environment_id=UUID('2ed5a523-961e-4984-a109-9e327a92efa3'), created_at=datetime.datetime(2025, 6, 19, 9, 57, 58, tzinfo=datetime.timezone.utc))

In [35]:
await permit.check(f"{user.id}", "read", 'document')

True